In [35]:
import numpy as np
from scipy.spatial.transform import Rotation

In [36]:
euler_angles = [30, 45, 60] 
r = Rotation.from_euler('xyz', euler_angles, degrees=True)
R = r.as_matrix()
print("旋转矩阵 R:")
print(R)

旋转矩阵 R:
[[ 0.35355339 -0.5732233   0.73919892]
 [ 0.61237244  0.73919892  0.28033009]
 [-0.70710678  0.35355339  0.61237244]]


In [37]:
orthogonality = np.dot(R.T, R)
print("验证正交性 R^T * R (应该接近单位矩阵):")
print(orthogonality)
print(f"是否正交: {np.allclose(orthogonality, np.eye(3))}")

验证正交性 R^T * R (应该接近单位矩阵):
[[1.00000000e+00 6.05844194e-17 1.61661354e-17]
 [6.05844194e-17 1.00000000e+00 6.89457359e-17]
 [1.61661354e-17 6.89457359e-17 1.00000000e+00]]
是否正交: True


In [38]:
det_R = np.linalg.det(R)
print(f"验证行列式 det(R) = {det_R:.6f} (应该接近1)")
print(f"是否为有效旋转矩阵: {np.isclose(det_R, 1.0)}")
print()

验证行列式 det(R) = 1.000000 (应该接近1)
是否为有效旋转矩阵: True



In [39]:
T = np.array([[0.1], 
              [0.2], 
              [0.3]])
def skew_symmetric(v):
    return np.array([[0, -v[2, 0], v[1, 0]],
                     [v[2, 0], 0, -v[0, 0]],
                     [-v[1, 0], v[0, 0], 0]])
T_skew = skew_symmetric(T)
print("反对称矩阵 [T]*:")
print(T_skew)
print("验证反对称性 [T]× + [T]×^T (应该为零矩阵):")
print(T_skew + T_skew.T)
print(f"是否反对称:{np.allclose(T_skew,-T_skew.T)}")

反对称矩阵 [T]*:
[[ 0.  -0.3  0.2]
 [ 0.3  0.  -0.1]
 [-0.2  0.1  0. ]]
验证反对称性 [T]× + [T]×^T (应该为零矩阵):
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
是否反对称:True


In [40]:
E = np.dot(T_skew,R)
print(E)

[[-0.32513309 -0.151049    0.03837546]
 [ 0.1767767  -0.20732233  0.16052243]
 [-0.00947343  0.18856455 -0.11980678]]


In [41]:
singular_values = np.linalg.svd(E, compute_uv=False)
print(f"本质矩阵的奇异值: {singular_values}")
print(f"前两个奇异值是否相等: {np.isclose(singular_values[0], singular_values[1])}")
print(f"第三个奇异值是否接近0: {np.isclose(singular_values[2], 0, atol=1e-10)}")

本质矩阵的奇异值: [3.74165739e-01 3.74165739e-01 1.03228665e-17]
前两个奇异值是否相等: True
第三个奇异值是否接近0: True


In [42]:
U, S, Vt = np.linalg.svd(E)
print("奇异值:", S)

奇异值: [3.74165739e-01 3.74165739e-01 1.03228665e-17]


In [43]:
W = np.array([[0, -1, 0],
              [1, 0, 0],
              [0, 0, 1]])
R1 = np.dot(np.dot(U, W), Vt)
R2 = np.dot(np.dot(U, W.T), Vt)

if np.linalg.det(R1) < 0:
    R1 = -R1
if np.linalg.det(R2) < 0:
    R2 = -R2

T1 = U[:, 2:3]
T2 = -U[:, 2:3]

print("分解的R1与原始R一致?", np.allclose(R, R1, atol=1e-6))
print("分解的R2与原始R一致?", np.allclose(R, R2, atol=1e-6))


分解的R1与原始R一致? False
分解的R2与原始R一致? True


In [44]:
T_norm = T / np.linalg.norm(T)
T1_norm = T1 / np.linalg.norm(T1)
T2_norm = T2 / np.linalg.norm(T2)

print("分解的T1方向与原始T一致?", np.allclose(T_norm, T1_norm, atol=1e-6))
print("分解的T2方向与原始T一致?", np.allclose(T_norm, T2_norm, atol=1e-6))


分解的T1方向与原始T一致? True
分解的T2方向与原始T一致? False


In [45]:
R_recovered = R1 if np.allclose(R, R1, atol=1e-6) else R2
T_recovered = T1_norm if np.allclose(T_norm, T1_norm, atol=1e-6) else T2_norm
T_recovered_skew = np.array([[0, -T_recovered[2,0], T_recovered[1,0]],
                             [T_recovered[2,0], 0, -T_recovered[0,0]],
                             [-T_recovered[1,0], T_recovered[0,0], 0]])
E_recovered = np.dot(T_recovered_skew, R_recovered)
E_norm = E / np.linalg.norm(E)
E_recovered_norm = E_recovered / np.linalg.norm(E_recovered)

print("重构的E与原始E一致?", 
      np.allclose(E_norm, E_recovered_norm, atol=1e-5) or 
      np.allclose(E_norm, -E_recovered_norm, atol=1e-5))

重构的E与原始E一致? True
